<a href="https://colab.research.google.com/github/anahimaciel/Networks-in-cities/blob/main/UserFilterAuto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
#takes 2m44s to run without caucaia
for city in os.listdir("/content/drive/MyDrive/Networks in cities/Dados"):
  if(city=="Fortaleza" or city==".ipynb_checkpoints"):
    continue

  #if(city!= "Caucaia"):
  #  continue

  print(city)

  #read csv
  df=pd.read_csv("/content/drive/MyDrive/Networks in cities/Dados/" +city+ "/cdr_"+ city +".txt", header=None, sep=';',usecols=[0,1,4,6,7])

  #cdr_data.columns = ["date", "time", "duration", "ddd_from",
  #"user_from", "ddd_to", "user_to", "antenna", "cell_id_to",
  # "nu_trafego", "tp_trafego", "hold_from", "hold_to", "tp_line", "unk_number"]

  df.columns = ["date", "time","user_from","user_to","antenna"]

  #create dataframe with number of calls of each user
  calls_by_user=df.groupby('user_from').agg({'date':'nunique','time':'nunique'}).reset_index()

  #drop users outside that made too few or too many calls and whoo called in less than 5 different days
  filter=((calls_by_user['date']>5) & (calls_by_user['time']>=10) & (calls_by_user['time']<=300))
  filtered_users= calls_by_user.loc[filter,'user_from']
  filtered_df=df[~df['user_from'].isin(filtered_users)]
  #aux=df['user_from'].map(calls_by_user)
  #filtered_df=df[(aux['time'].between(5,300, inclusive=True)) & (not aux['date'].between(0,4, inclusive=True))]

  #saving demography data
  with open("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/demo.txt",'a') as file:
    file.truncate(0)
    file.write("users: "+str(len(calls_by_user))+"\n")
    file.write("significative users: "+str(len(filtered_users))+"\n")
    file.write("calls: " + str(len(df))+ "\n")
    file.write("calls by sig users: "+str(len(filtered_df))+"\n")

  #saving result
  filtered_df.to_csv("/content/drive/MyDrive/Networks in cities/Dados/"+city+"/filtered_cdr.txt")